In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, split
from itertools import combinations

In [ ]:
from google.colab import files
files.upload()


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"daryasavitskaya","key":"32123b65cc42883a72a236b0c552fca2"}'}

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "daryasavitskaya"
os.environ['KAGGLE_KEY'] = "****"

!pip install kaggle
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024
!unzip 1-3m-linkedin-jobs-and-skills-2024.zip -d ./data/

Dataset URL: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024
License(s): ODC Attribution License (ODC-By)
 99% 1.86G/1.88G [00:27<00:00, 120MB/s]
100% 1.88G/1.88G [00:27<00:00, 73.0MB/s]
Archive:  1-3m-linkedin-jobs-and-skills-2024.zip
  inflating: ./data/job_skills.csv   
  inflating: ./data/job_summary.csv  
  inflating: ./data/linkedin_job_postings.csv  


In [54]:
spark = SparkSession.builder.appName("PCY Algorithm").getOrCreate()

In [55]:
df = spark.read.csv('./data/job_skills.csv', header=True, inferSchema=True)
df = df.dropna(subset=['job_skills'])
df = df.withColumn('job_skills', lower(col('job_skills')))

In [56]:
rdd = df.rdd.map(lambda row: row.job_skills.split(","))

In [57]:
support = 1200
buckets = 2000000

In [58]:
frdd = rdd.flatMap(lambda basket: [(skill.strip(), 1) for skill in basket]) \
          .reduceByKey(lambda x, y: x + y) \
          .filter(lambda x: x[1] > support)

frequent = set(frdd.map(lambda x: x[0]).collect())
print(f"it:1, frequents:{len(frequent)}, samples:{frdd.take(3)}")

fbuckets = rdd.flatMap(lambda basket: [(hash(tuple(sorted(pair))) % buckets, 1)
                                       for pair in combinations(basket, 2)]) \
              .reduceByKey(lambda x, y: x + y) \
              .map(lambda x: (x[0], int(x[1] > support)))

def from_buckets_to_bitmap(fbuckets, num_buckets):
    bitmap = [0] * num_buckets
    for (bucket, count) in fbuckets.collect():
        bitmap[bucket] = count
    return bitmap

bitmap = from_buckets_to_bitmap(fbuckets, buckets)
print(f"Bitmap size: {len(bitmap)}, Sum of bitmap: {sum(bitmap)}")

buckets_data = fbuckets.collect()
print(f"Some buckets: {buckets_data[:10]}")

k = 2
while True:
    frdd = rdd.flatMap(lambda basket: [(tuple(sorted(comb)), 1)
                                       for comb in combinations(basket, k)
                                       if all([subset in frequent for subset in combinations(comb, k - 1)])]) \
              .filter(lambda x: bitmap[hash(x[0]) % buckets]) \
              .reduceByKey(lambda x, y: x + y) \
              .filter(lambda x: x[1] > support)

    frequent = set(frdd.map(lambda x: x[0]).collect())

    if len(frequent) == 0:
        break

    print(f"it:{k}, frequents:{len(frequent)}, all frequents:{list(frdd.collect())}")


    fbuckets = rdd.flatMap(lambda basket: [(hash(tuple(sorted(comb))) % buckets, 1)
                                           for comb in combinations(basket, k + 1)]) \
                  .reduceByKey(lambda x, y: x + y) \
                  .map(lambda x: (x[0], int(x[1] > support)))

    bitmap = from_buckets_to_bitmap(fbuckets, buckets)
    print(f"Bitmap updated: k={k+1}, Bitmap sum: {sum(bitmap)}")

    k += 1

spark.stop()

it:1, frequents:2200, samples:[('waste management', 1574), ('attention to detail', 133929), ('high school diploma', 67244)]
Bitmap size: 2000000, Sum of bitmap: 11158
Some buckets: [(1306722, 0), (1397532, 0), (1927524, 0), (702552, 0), (99192, 0), (1460220, 0), (627582, 0), (1195416, 0), (1047306, 1), (879654, 0)]
